In [1]:
import sys
import os
import shutil
import subprocess
extra_path = "/Users/lraming/Documents/Repos/GP4" # Path to tribsmodel.py 
if extra_path not in sys.path:
    sys.path.append(extra_path)

os.chdir("../..")

In [2]:
from tribsmodel import Model as model

In [3]:
m = model()

__Meshbuilder can be a bit messy:__ but all it needs is the points file for the mesh specified in the .in file and a results path that exist. The strategy then is to create the directory were the graph files will be saved, copy mesh builder into the file, and the .in file, then run mesh builder. Using os.abs path should be helpful here. 

In [4]:

# Define the relative path to save mesh builder results
path = "data/model/parallel_mesh/sycMS4/"
points = "data/model/points/MS4s1z1_tree5_1.points"

## USE this to add subdirectories if they do not exist
# Split the full path into individual directory names
directories = path.split('/')

# Initialize the base directory
mb_results = ''

# Create the directories one by one
for directory in directories:
    mb_results = os.path.join(mb_results, directory)
    try:
        os.mkdir(mb_results)
    except FileExistsError:
        # Directory already exists, so continue without raising an error
        pass

In [5]:
mb_results

'data/model/parallel_mesh/sycMS4/'

In [6]:
# read input file
base_name = 'ms3_'
m.read_input_file("src/infiles/template.in")
m.options["outfilename"]["value"] = os.path.abspath(mb_results)
m.options["optmeshinput"]["value"] = 9 # infamous option 9
m.options["pointfilename"]["value"] = os.path.abspath(points)

In [7]:
# move mesh builder into the folder where the graph file will be saved
shutil.copy("bin/meshBuilder/meshBuilder",mb_results)

'data/model/parallel_mesh/sycMS4/meshBuilder'

In [18]:
infile = "sycMS3.in"
os.chdir("../sycMS3")

In [ ]:
m.write_input_file(infile)

In [9]:
command = ["./meshBuilder", infile]

In [10]:
subprocess.run(command)

Reading input file sycMS4.in
bMeshBuilder: Start ... Tue Oct 24 15:50:13 2023

bMesh: MakeBasicMesh ... Tue Oct 24 15:50:13 2023


Reading in '/Users/lraming/Documents/Repos/Sycamore/data/model/points/MS4s1z1_tree5_1.points' points file...
Number of points to read: 69602
Read in 69602 points 
Number of regular points 62311
Number of stream points 4309
Number of closed boundary points 2981
Number of open boundary points 1
Creating Node Lookup Table

Computing triangulation...elapsed time (time)= 1 s (clock)= 0.183064 s

Creating edge list...
Triangulation building spokes
Creating Edge Lookup Table
Adding spokes to nodes and writing temporary nodes file
Building counter clockwise edge pointer in edges
Write temporary edge file
Triangulation building element table
nelem = 139166 npoints = 69602 nedges = 208767
Created elems...i
Created edges_visit...
ielem = 139166 nelem = 139166
Write temporary triangles file
Finished building basic mesh and writing to files
bMesh: RefineBasicMesh ... Tu

CompletedProcess(args=['./meshBuilder', 'sycMS4.in'], returncode=0)

__Now for the second step__, the connectivity files from the mesh builder need to be assembelled using a shell script that relies on perl and gpmetis. Currently the you have to assmeble metis seperately, but maybe could add it to the meshbuilder build. (also need to add helper scripts). For now I will manually copy those over but all that should be automated if possible. 

In [21]:
os.chdir('../sycMS3/')


In [26]:
# command to be passed to subprocess: number of nodes, option for partioning, base name of simulation
infile = "sycMS3.in"
num_processors = 8
opt = 1  # surface connectivity
command = ["./run_metis.zsh", str(num_processors) , str(opt) ,infile]
subprocess.run(command)

CompletedProcess(args=['./run_metis.zsh', '8', '1', 'sycMS3.in'], returncode=0)